In [2]:
#@title setup packages

# Import necessary modules
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
import bs4


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Loading Dataset

In [7]:
# Loading and storing datasets
# Load documents (e.g., from local storage)
#document_loader = DocumentLoader("path_to_local_storage")

#Loading from web
# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

# Splitting dataset

In [14]:
#Define different text splitter methods
# Recursive Character Text Splitter
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from  langchain_community.document_loaders.pdf import PyPDFLoader

all_docs = []
all_splits = []

pdf_directory = "./"

recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20, add_start_index=True)

for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdfloader = PyPDFLoader(os.path.join(pdf_directory,filename))
        docs = pdfloader.load_and_split(text_splitter=recursive_splitter)
        
        for doc in docs:
            all_docs.append(doc)
        


# HTML Header Text Splitter
#html_splitter = HTMLHeaderTextSplitter()

# Markdown Header Text Splitter
#markdown_splitter = MarkdownHeaderTextSplitter()

# Code Text Splitter
#code_splitter = CodeTextSplitter()

# Token Text Splitter
#token_splitter = TokenTextSplitter()

# Character Text Splitter
#character_splitter = CharacterTextSplitter()

# Semantic Chunker
#semantic_chunker = SemanticChunker()

# AI21 Semantic Text Splitter
#ai21_splitter = AI21SemanticTextSplitter()

Embeddings

In [23]:

# Define documents

from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

sentence_transformer_ef = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

embedding_function = sentence_transformer_ef 

# Embed and store document splits
vectorstore = Chroma.from_documents(documents=all_docs, embedding=embedding_function)


# Other embedding models are available in:
# - lang_chain_embedding_models
# - sentence-transformers
# - Kaggle (includes all ML models, not just embedding ones)

# Retrieving top k relevant embeddings
def retrieve_top_k(vectorstore, query, k=5, method='cosine'):
    if method == 'cosine':
        return vectorstore.retrieve(query, k=k, method='cosine')
    elif method == 'dot_product':
        return vectorstore.retrieve(query, k=k, method='dot_product')
    elif method == 'euclidean':
        return vectorstore.retrieve(query, k=k, method='euclidean')
    else:
        raise ValueError("Unsupported retrieval method")



LLM

In [ ]:
# Generate final answers
from langchain import LLM
# Choose an LLM model
llm = LLM()

# Define RAG Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
   {"context": retriever | format_docs, "question": RunnablePassthrough()}
   | prompt
   | llm
   | StrOutputParser()
)

# Example usage of the RAG chain
for chunk in rag_chain.stream("What is Task Decomposition?"):
   print(chunk, end="", flush=True)

# The prompt is expected to be a dict with keys "context" and "question".
# retriever | format_docs passes the question through the retriever, generating Document objects, and then to format_docs to generate strings;
# RunnablePassthrough() passes through the input question unchanged;
# llm runs the inference;
# StrOutputParser() plucks the string content out of the LLM's output message.

# Example usage of the retriever
query = "What is Task Decomposition?"
top_k_docs = retrieve_top_k(vectorstore, query)

for doc in top_k_docs:
    print(doc.page_content)
